<h1>Separating JSON Data into columns (Credits)</h1>

In [1]:
import csv, json, gensim
from gensim import corpora

rowslist = []
departmentslist = []
jobtitleslist = []

#read file
with open('tmdb_5000_credits.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else: #if line_count != 5:
            eachnewrow = []
            departments = []
            jobtitles = []
                  
            malecastsnum = 0
            femalecastsnum = 0
            unknowngendercastsnum = 0    
            totalcasts = 0

            malecrewsnum = 0
            femalecrewsnum = 0
            unknowngendercrewsnum = 0
            totalcrews = 0
                  
            eachnewrow.append(row[0]) #movie_id
            eachnewrow.append(row[1]) #title

            #handle json casts
            jsonstring = row[2]
            jsonobj = json.loads(jsonstring)
            
            for jsonelement in jsonobj:
                gender = jsonelement['gender']       
                if gender == 0:
                  unknowngendercastsnum += 1
                elif gender == 1:
                  femalecastsnum += 1
                elif gender == 2:
                  malecastsnum += 1
                
                totalcasts += 1
            
            eachnewrow.append(malecastsnum) #malecastsnum
            eachnewrow.append(femalecastsnum) #femalecastsnum                  
            eachnewrow.append(unknowngendercastsnum) #unknowngendercastsnum 
            eachnewrow.append(totalcasts) #totalcasts                   
                  
            #handle json crews              
            jsonstring2 = row[3]
            jsonobj2 = json.loads(jsonstring2)
            
            for jsonelement in jsonobj2:
                gender = jsonelement['gender']       
                if gender == 0:
                  unknowngendercrewsnum += 1
                elif gender == 1:
                  femalecrewsnum += 1
                elif gender == 2:
                  malecrewsnum += 1  
                  
                totalcrews += 1
                  
                department = jsonelement['department']
                departments.append(department.lower())         
                  
                jobtitle = jsonelement['job']
                jobtitles.append(jobtitle.lower())                    

            eachnewrow.append(malecrewsnum) #malecastsnum
            eachnewrow.append(femalecrewsnum) #femalecastsnum                  
            eachnewrow.append(unknowngendercrewsnum) #unknowngendercastsnum                   
            eachnewrow.append(totalcrews) #totalcrews  
                  
            eachnewrow.append(departments) #departments
            departmentslist.append(departments)  
                  
            eachnewrow.append(jobtitles) #jobtitles
            jobtitleslist.append(jobtitles)
                  
            rowslist.append(eachnewrow)
                  
            line_count += 1

    print(f'Processed {line_count} lines.')
                  
departmentsdictionary = corpora.Dictionary(departmentslist)
token_to_id_departments = departmentsdictionary.token2id
#print(f'Dictionary for Departments: {token_to_id_departments}') 
#print(f'\nNo. of Departments: {len(token_to_id_departments)}')
                  
jobtitlesdictionary = corpora.Dictionary(jobtitleslist)
token_to_id_jobtitles = jobtitlesdictionary.token2id
#print(f'Dictionary for Job Titles: {token_to_id_jobtitles}')        
#print(f'No. of Job Titles: {len(token_to_id_jobtitles)}')                   

def assigningValues(vector, listofzeros):
    for counter, value in enumerate(vector): 
        index = 0       
        for value2 in listofzeros:                      
            if index == value[0]:
                listofzeros[index] = value[1]    
            index += 1
                  
    return listofzeros                  
                  
#write to file
with open('new_tmdb_5000_credits.csv', mode='w', newline='') as credits_file:
    writer = csv.writer(credits_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    writer.writerow(['movie_id', 'title', 'malecastsnum', 'femalecastsnum', 'unknowngendercastsnum', 'totalcasts', 'malecrewsnum', 'femalecrewsnum', 'unknowngendercrewsnum', 'totalcrews'] 
        + list(departmentsdictionary.values()) + list(jobtitlesdictionary.values()))              
    
    for eachrow in rowslist:
        listofzerosdepartments = [0] * len(token_to_id_departments)
        listofzerosjobtitles = [0] * len(token_to_id_jobtitles)                         
                  
        vec_departments = departmentsdictionary.doc2bow(eachrow[10])
        vec_jobtitles = jobtitlesdictionary.doc2bow(eachrow[11])   

        listofzerosdepartments = assigningValues(vec_departments, listofzerosdepartments)
        listofzerosjobtitles = assigningValues(vec_jobtitles, listofzerosjobtitles)

        writer.writerow(eachrow[0:10] + listofzerosdepartments + listofzerosjobtitles)

    print('\nOutput Success!')
                  

C:\Users\andy_\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Column names are movie_id, title, cast, crew
Processed 4804 lines.

Output Success!


<h1>Separating JSON Data into columns (Movies)</h1>

In [8]:
import csv, json, gensim, datetime, time
from gensim import corpora

rowslist = []
genreslist = []
#productioncompanieslist = []
#productioncountrieslist = []

def readJSON(row, rownumber, list, keyword):
    jsonstring = row[rownumber]
    jsonobj = json.loads(jsonstring)

    for jsonelement in jsonobj:
        keywordelement = jsonelement[keyword]       
        list.append(keywordelement.lower()) 
    
    return list
    
#read file
with open('tmdb_5000_movies.csv', encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else: #if line_count != 5:
            eachnewrow = []
            genres = []
            productioncompanies = []
            productioncountries = []
                  
            eachnewrow.append(row[0]) #budget

            #handle json genres
            genres = readJSON(row, 1, genres, 'name')
                  
            eachnewrow.append(genres) #genres
            genreslist.append(genres)
            
            #row[2] homepage removed
            eachnewrow.append(row[3]) #id
                        
            #row[4] keywords removed
            #row[5] original_language removed
            #row[6] original_title removed  
            #row[7] overview removed                    
            eachnewrow.append(row[8]) #popularity                  

            #handle json productioncompanies
            productioncompanies = readJSON(row, 9, productioncompanies, 'name')
                  
            eachnewrow.append(len(productioncompanies)) #productioncompanies
            #productioncompanieslist.append(productioncompanies)                  
 
            #handle json productioncountries
            productioncountries = readJSON(row, 10, productioncountries, 'name')
                  
            eachnewrow.append(len(productioncountries)) #productioncountries
            #productioncountrieslist.append(productioncountries) 
            
            weekday = ""
            day = ""
            month = ""
            year = ""                 
                  
            #handle releasedate 
            if row[11] != "":
                d = datetime.datetime.strptime(row[11] ,'%Y-%m-%d')

                weekdayword = d.strftime('%a')
                weekday = time.strptime(weekdayword, '%a').tm_wday + 1
                
                day = d.strftime('%d')
                
                monthword = d.strftime('%b')
                month = time.strptime(monthword,'%b').tm_mon

                year = d.strftime('%Y')
                
            eachnewrow.append(weekday) #weekday
            eachnewrow.append(day) #day 
            eachnewrow.append(month) #month
            eachnewrow.append(year) #year
                  
            eachnewrow.append(row[12]) #revenue
            eachnewrow.append(row[13]) #runtime
            #row[14] spoken_languages removed  
            #row[15] status removed 
            #row[16] tagline removed   
            eachnewrow.append(row[17]) #title
            eachnewrow.append(row[18]) #vote_average
            eachnewrow.append(row[19]) #vote_count                  
            
            WR = (float(row[19]) / (float(row[19])+25000)) * float(row[18]) + (25000 / (float(row[19]) + 25000)) * 7.0            
            eachnewrow.append(WR) #weighted_rating
            
            rowslist.append(eachnewrow)
                  
            line_count += 1

    print(f'Processed {line_count} lines.')   
                  
genresdictionary = corpora.Dictionary(genreslist)
token_to_id_genres = genresdictionary.token2id                

#productioncompaniesdictionary = corpora.Dictionary(productioncompanieslist)
#token_to_id_productioncompanies = productioncompaniesdictionary.token2id
                  
#productioncountriesdictionary = corpora.Dictionary(productioncountrieslist)
#token_to_id_productioncountries = productioncountriesdictionary.token2id   

def assigningValues(vector, listofzeros):
    for counter, value in enumerate(vector): 
        index = 0       
        for value2 in listofzeros:                      
            if index == value[0]:
                listofzeros[index] = value[1]    
            index += 1
                  
    return listofzeros
                  
#write to file
with open('new_tmdb_5000_movies.csv', mode='w', newline='', encoding='utf-8') as movies_file:
    writer2 = csv.writer(movies_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    writer2.writerow(['budget'] + list(genresdictionary.values()) + ['id', 'popularity', 'production_companies', 'production_countries'] 
        #+ list(productioncompaniesdictionary.values()) + list(productioncountriesdictionary.values()) 
        + ['weekday', 'day', 'month', 'year', 'revenue', 'runtime', 'title', 'vote_average', 'vote_count', 'weighted_rating'])              
    
    for eachrow in rowslist:
        listofzerosgenres = [0] * len(token_to_id_genres)
        #listofzerosproductioncompanies = [0] * len(token_to_id_productioncompanies)
        #listofzerosproductioncountries = [0] * len(token_to_id_productioncountries)                   
                  
        vec_genres = genresdictionary.doc2bow(eachrow[1])
        #vec_productioncompanies = productioncompaniesdictionary.doc2bow(eachrow[4])
        #vec_productioncountries = productioncountriesdictionary.doc2bow(eachrow[5])                   

        listofzerosgenres = assigningValues(vec_genres, listofzerosgenres)
        #listofzerosproductioncompanies = assigningValues(vec_productioncompanies, listofzerosproductioncompanies)
        #listofzerosproductioncountries = assigningValues(vec_productioncountries, listofzerosproductioncountries)

        writer2.writerow(eachrow[0:1] + listofzerosgenres + eachrow[2:6] 
            #+ listofzerosproductioncompanies + listofzerosproductioncountries 
            + eachrow[6:16])
                  
    print('\nOutput Success!')                 

Column names are budget, genres, homepage, id, keywords, original_language, original_title, overview, popularity, production_companies, production_countries, release_date, revenue, runtime, spoken_languages, status, tagline, title, vote_average, vote_count
Processed 4804 lines.

Output Success!


<h1>Merging two files</h1>

In [10]:
import pandas as pd

movies = pd.read_csv("new_tmdb_5000_movies.csv")
credits = pd.read_csv("new_tmdb_5000_credits.csv")
merged = pd.merge(movies, credits, left_on=['id'],
                   right_on=['movie_id'], 
                   how='left')
merged.to_csv("train.csv", index=False)

print('\nOutput Success!')  


Output Success!
